In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [2]:
BASE_PATH = "../data/processed"
TRAIN_PATHS = [
    "X_train_con_outliers.xlsx",
    "X_train_sin_outliers.xlsx",
    "X_train_con_outliers_norm.xlsx",
    "X_train_sin_outliers_norm.xlsx",
    "X_train_con_outliers_scal.xlsx",
    "X_train_sin_outliers_scal.xlsx"
]
TRAIN_DATASETS = []
for path in TRAIN_PATHS:
    TRAIN_DATASETS.append(
        pd.read_excel(os.path.join(BASE_PATH, path))
    )

TEST_PATHS = [
    "X_test_con_outliers.xlsx",
    "X_test_sin_outliers.xlsx",
    "X_test_con_outliers_norm.xlsx",
    "X_test_sin_outliers_norm.xlsx",
    "X_test_con_outliers_scal.xlsx",
    "X_test_sin_outliers_scal.xlsx"
]
TEST_DATASETS = []
for path in TEST_PATHS:
    TEST_DATASETS.append(
        pd.read_excel(os.path.join(BASE_PATH, path))
    )

y_train = pd.read_excel(os.path.join(BASE_PATH, "y_train.xlsx"))
y_test = pd.read_excel(os.path.join(BASE_PATH, "y_test.xlsx"))

In [4]:
#Acertado

import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold
import json

# Asegurarse de que y_train y y_test sean vectores 1D
y_train = y_train.values.ravel()  # Convertir a array de NumPy y luego a 1D
y_test = y_test.values.ravel()    # Convertir a array de NumPy y luego a 1D

# Crear directorio para guardar los modelos si no existe
os.makedirs("../models/", exist_ok=True)

# Lista para almacenar resultados
results = []

# Definir el número de características que deseas seleccionar
k = 5  # Puedes cambiar este valor según el número de características que desees seleccionar

for index, dataset in enumerate(TRAIN_DATASETS):
    print(f"Procesando dataset {index + 1}")

    # Verificar que dataset sea un DataFrame
    if not isinstance(dataset, pd.DataFrame):
        dataset = pd.DataFrame(dataset)

    # Eliminar características constantes
    selector_constant = VarianceThreshold()
    X_train_no_constant = selector_constant.fit_transform(dataset)
    X_test_no_constant = selector_constant.transform(TEST_DATASETS[index])

    # Selección de características (usando SelectKBest)
    selector = SelectKBest(f_classif, k=k)
    X_train_selected = selector.fit_transform(X_train_no_constant, y_train)
    X_test_selected = selector.transform(X_test_no_constant)

    # Obtener los nombres de las características seleccionadas
    non_constant_features = dataset.columns[selector_constant.get_support()].tolist()
    selected_features = [non_constant_features[i] for i in selector.get_support(indices=True)]

    # Guardar las características seleccionadas en un archivo JSON
    selected_features_filename = f"selected_features_{index + 1}k{k}.json"
    with open(f"../models/{selected_features_filename}", 'w') as json_file:
        json.dump(selected_features, json_file)

    # Entrenamiento del modelo con las características seleccionadas
    model = LogisticRegression(random_state=42, max_iter=10000)
    model.fit(X_train_selected, y_train)
    
    # Predicciones
    y_pred_train = model.predict(X_train_selected)
    y_pred_test = model.predict(X_test_selected)

    # Almacenar los resultados de precisión
    results.append(
        {
            "train": accuracy_score(y_train, y_pred_train),
            "test": accuracy_score(y_test, y_pred_test)
        }
    )

# Imprimir los resultados
for i, result in enumerate(results):
    print(f"Dataset {i+1} - Train Accuracy: {result['train'] * 100:.4f}%, Test Accuracy: {result['test'] * 100:.4f}%")

Procesando dataset 1
Procesando dataset 2
Procesando dataset 3
Procesando dataset 4
Procesando dataset 5
Procesando dataset 6
Dataset 1 - Train Accuracy: 90.4281%, Test Accuracy: 89.8616%
Dataset 2 - Train Accuracy: 90.2580%, Test Accuracy: 89.7523%
Dataset 3 - Train Accuracy: 90.4189%, Test Accuracy: 89.8859%
Dataset 4 - Train Accuracy: 90.2520%, Test Accuracy: 89.7402%
Dataset 5 - Train Accuracy: 90.2793%, Test Accuracy: 89.7402%
Dataset 6 - Train Accuracy: 90.2338%, Test Accuracy: 89.7887%


In [ ]:
import json
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as np 

# Asegurarse de que y_train y y_test sean vectores 1D
y_train = y_train.ravel()  # Convertir a array de NumPy y luego a 1D
y_test = y_test.ravel()    # Convertir a array de NumPy y luego a 1D

# Mejor dataset (esto lo debes definir según el conjunto de datos que hayas seleccionado como el mejor)
best_dataset_index = 2  # Cambia este valor con el índice del mejor dataset que hayas seleccionado

# Define tu modelo y el grid search
model = LogisticRegression(random_state=42, solver='sag', tol=0.1)

# Parámetros para GridSearchCV (esto es solo un ejemplo, ajusta según tus necesidades)
param_grid = {
    "C": np.logspace(-3, 3, 5),  # Reducir a 5 valores para C
    "penalty": ["l1", "l2"],  # Probar solo l1 y l2
    "solver": ['liblinear', 'saga'],  # Limitar a 'liblinear' y 'saga', que son más rápidos
    "fit_intercept": [True, False],  # Mantener el ajuste de la intercepción
    "dual": [False],  # 'dual=True' solo con 'liblinear', por lo que se mantiene en False
    "class_weight": [None, "balanced"],  # Probar con 'None' y 'balanced'
    "multi_class": ['auto', 'ovr'],  # Reducir a combinaciones más comunes
    'tol': [1e-4, 1e-3],  # Reducir a solo dos valores para tolerancia
    'max_iter': [100, 500],  # Reducir el número de iteraciones a 2 opciones
    'warm_start': [True, False],  # Continuar desde la última solución
    'l1_ratio': np.linspace(0, 1, 3)  # Reducir a 3 valores para l1_ratio
}

# GridSearchCV con menos combinaciones de parámetros
grid = GridSearchCV(
    model, param_grid=param_grid, cv=3,  # Usar 3 pliegues para validación cruzada
    n_jobs=-1,  # Utilizar todos los núcleos del procesador
    verbose=3,  # Mayor nivel de detalle en la salida de la búsqueda
    scoring='accuracy',  # Optimizar para precisión
)

# Ajustamos el grid search al conjunto de datos seleccionado
grid.fit(TRAIN_DATASETS[best_dataset_index], y_train)

# Obtener el mejor modelo después de GridSearchCV
final_model = grid.best_estimator_

# Predicciones sobre el conjunto de entrenamiento
y_pred_train = final_model.predict(TRAIN_DATASETS[best_dataset_index])

# Predicciones sobre el conjunto de prueba
y_pred_test = final_model.predict(TEST_DATASETS[best_dataset_index])

# Calcular la exactitud
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Guardar los hiperparámetros y las métricas en un archivo JSON
data_to_save = {
    "hyperparams": grid.best_params_,  # Los mejores hiperparámetros encontrados
    "train_accuracy": train_accuracy,
    "test_accuracy": test_accuracy
}

# Guardamos los resultados en un archivo JSON
with open("../models/hyperparams_best_dataset.json", "w") as f:
    json.dump(data_to_save, f, indent=4)

# Imprimir las métricas de rendimiento
print(f"Training Accuracy: {train_accuracy * 100:.4f}")
print(f"Test Accuracy: {test_accuracy * 100:.4f}")


AttributeError: 'numpy.ndarray' object has no attribute 'values'